In [22]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as sl
import scipy as sp

For the majority of differential equations, they are generally sparse, meaning that most entries are zero (the alternative is termed 'dense'). 

We can devise more efficient storage methods i.e. by skipping over operations that we know involve multiplications by zero as we know the answer will be zero

For Jacobi and Gauss-Seidel, the cost of each iteration is dependent on the number of unknowns n, so computational cost of methods scales as n^2
If the row only contains a fixed, small number of non-zero entries, we can simply skip the zero entries and cost per iteration becomes linear in n

In [23]:
n = 50
main_diag = np.ones(n) #creates a vector of ones 

off_diag = np.random.random(n-1)
A = np.diag(-2*main_diag, 0) + np.diag(1*off_diag,1) + np.diag(1*off_diag,-1)
#A random RHS vector
b = np.random.random(A.shape[0])
print(A)

sA = sp.sparse.csr_matrix(A)
print('This is the same matrix but now stored in a sparse matrix data structure.')
print(sA)

[[-2.          0.17138084  0.         ...  0.          0.
   0.        ]
 [ 0.17138084 -2.          0.09678585 ...  0.          0.
   0.        ]
 [ 0.          0.09678585 -2.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... -2.          0.43715712
   0.        ]
 [ 0.          0.          0.         ...  0.43715712 -2.
   0.15080316]
 [ 0.          0.          0.         ...  0.          0.15080316
  -2.        ]]
This is the same matrix but now stored in a sparse matrix data structure.
  (0, 0)	-2.0
  (0, 1)	0.17138083832092
  (1, 0)	0.17138083832092
  (1, 1)	-2.0
  (1, 2)	0.09678584523742062
  (2, 1)	0.09678584523742062
  (2, 2)	-2.0
  (2, 3)	0.06387700873888613
  (3, 2)	0.06387700873888613
  (3, 3)	-2.0
  (3, 4)	0.8992846758850166
  (4, 3)	0.8992846758850166
  (4, 4)	-2.0
  (4, 5)	0.5151022225036674
  (5, 4)	0.5151022225036674
  (5, 5)	-2.0
  (5, 6)	0.7463709953190173
  (6, 5)	0.7463709953190173
  (6, 6)	-2.0
  (6, 7)	0.2602728387962431
  (7

We are now using a Conjugate Gradient algorithm from scipy to define a callback function which we can pass to the solver
This will store and print the iteration numbers and residuals, basically a method to output some information on the solver

In [25]:
def gen_callback_cg():
    diagnostics = dict(it=0, residuals=[]) 
    def callback(xk):   # xk is the solution computed by CG at each iteration
        diagnostics["it"] += 1
        diagnostics["residuals"].append(sl.norm(A @ xk - b))
        print(diagnostics["it"], sl.norm(A @ xk - b))
    return callback    

print('Execute the CG algorithm with:')
x_sol = sp.sparse.linalg.cg(A,b,x0=None, tol=1e-10, maxiter=1000, callback=gen_callback_cg())

Execute the CG algorithm with:
1 1.863970419953804
2 0.657245681808171
3 0.22586008545408187
4 0.08582838228317638
5 0.03245076142172889
6 0.012452529803414774
7 0.004718083825529428
8 0.0019625768394302455
9 0.0006267090857520234
10 0.0001635982467542102
11 7.205081705375431e-05
12 2.3276483777315176e-05
13 8.891211402249596e-06
14 3.3409904448683887e-06
15 8.208897670752107e-07
16 2.0396029205420728e-07
17 7.227537572898465e-08
18 2.408923233665921e-08
19 7.198343439450489e-09
20 2.6033476098151843e-09
21 7.909912374985051e-10
22 2.7135950572383623e-10
